In [3]:
import numpy as np

# Parameters
data_bits = np.array([1, 0, 1, 1, 0])  # Example data bits
chip_rate = 10  # Number of chips per data bit

# Generate PN code (example: simple binary PN code)
pn_code = np.random.choice([1, -1], size=chip_rate)

# Spread the data signal
spread_signal = np.repeat(data_bits, chip_rate) * np.tile(pn_code, len(data_bits))

# Simulate received signal (assume perfect transmission for simplicity)
received_signal = spread_signal

# Despread the received signal
despread_signal = np.sign(np.correlate(received_signal, np.tile(pn_code, len(data_bits)), mode='valid'))

# Recover original data bits
recovered_bits = (despread_signal > 0).astype(int)

print("Original data bits: ", data_bits)
print("Recovered data bits:", recovered_bits)


Original data bits:  [1 0 1 1 0]
Recovered data bits: [1]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.signal import correlate

# Load an audio file
audio_signal, sample_rate = sf.read('your_audio_file.wav')  # Replace with your audio file

# Parameters
chip_rate = 10  # Number of chips per audio sample

# Generate PN code
pn_code = np.random.choice([1, -1], size=chip_rate)

# Function to spread the audio signal
def spread_signal(audio_signal, pn_code, chip_rate):
    spread_signal = np.repeat(audio_signal, chip_rate) * np.tile(pn_code, len(audio_signal))
    return spread_signal

# Function to despread the signal
def despread_signal(received_signal, pn_code, chip_rate):
    despread_signal = np.correlate(received_signal, np.tile(pn_code, len(received_signal) // chip_rate), mode='valid')
    despread_signal = despread_signal[::chip_rate]
    return despread_signal

# Spread the audio signal
spread_audio_signal = spread_signal(audio_signal, pn_code, chip_rate)

# Simulate received signal (assume perfect transmission for simplicity)
received_signal = spread_audio_signal

# Despread the received signal
recovered_audio_signal = despread_signal(received_signal, pn_code, chip_rate)

# Normalize the recovered audio signal
recovered_audio_signal = np.interp(recovered_audio_signal, (recovered_audio_signal.min(), recovered_audio_signal.max()), (-1, 1))

# Save the recovered audio signal
sf.write('recovered_audio_file.wav', recovered_audio_signal, sample_rate)

# Plot the original, spread, and recovered signals
plt.figure(figsize=(15, 5))

plt.subplot(3, 1, 1)
plt.plot(audio_signal)
plt.title('Original Audio Signal')

plt.subplot(3, 1, 2)
plt.plot(spread_audio_signal)
plt.title('Spread Audio Signal')

plt.subplot(3, 1, 3)
plt.plot(recovered_audio_signal)
plt.title('Recovered Audio Signal')

plt.tight_layout()
plt.show()

# Play the original and recovered audio
import IPython.display as ipd

print("Original Audio:")
ipd.display(ipd.Audio(audio_signal, rate=sample_rate))

print("Recovered Audio:")
ipd.display(ipd.Audio(recovered_audio_signal, rate=sample_rate))
